#### 1. Import pandas library

In [1]:
import pandas as pd
import numpy as np

#### 2. Import users table:

In [2]:
#import users tab

doc = pd.read_csv("users_table.csv")
doc.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 3. Rename Id column to userId

In [3]:
ustb = doc.rename(columns={"Id": "userId"})
#doc
ustb

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40320,55743,1,2014-09-13 21:03:50,AussieMeg,2014-09-13 21:18:52,NaN,NaN,NaN,0,0,0,5026902,NaN,http://graph.facebook.com/665821703/picture?ty...
40321,55744,6,2014-09-13 21:39:30,Mia Maria,2014-09-13 21:39:30,NaN,NaN,NaN,1,0,0,5026998,NaN,NaN
40322,55745,101,2014-09-13 23:45:27,tronbabylove,2014-09-13 23:45:27,NaN,United States,NaN,0,0,0,481766,NaN,https://www.gravatar.com/avatar/faa7a3fdbd8308...
40323,55746,106,2014-09-14 00:29:41,GPP,2014-09-14 02:05:17,NaN,NaN,"<p>Stats noobie, product, marketing &amp; medi...",1,0,0,976289,NaN,https://www.gravatar.com/avatar/6d9e9fa6b783a3...


#### 4. Import posts table:

In [4]:
posts = pd.read_csv("posts_table.csv")
posts

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,48321,2,NaN,2013-01-23 09:00:01,0,NaN,<p>you can use the matlab codes for svm and co...,19966.0,2013-01-23 09:00:01,NaN,...,NaN,0,NaN,NaN,NaN,NaN,45118.0,NaN,NaN,NaN
39996,48322,2,NaN,2013-01-23 09:09:34,3,NaN,"<p>I use <a href=""http://www.gnu.org/software/...",892.0,2013-01-23 13:13:30,NaN,...,NaN,2,NaN,892.0,2013-01-23 13:13:30,NaN,48311.0,NaN,NaN,NaN
39997,48323,2,NaN,2013-01-23 09:16:44,1,NaN,"<p>If I understand your question correctly, yo...",2020.0,2013-01-23 09:16:44,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48247.0,NaN,NaN,NaN
39998,48324,2,NaN,2013-01-23 09:36:07,3,NaN,"<p>Doesn't really help you with your question,...",19914.0,2013-01-23 09:36:07,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48297.0,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [5]:
posts_tab = posts.rename(columns={"Id": "postId", "OwnerUserId": "userId"})
#posts
posts_tab

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,48321,2,NaN,2013-01-23 09:00:01,0,NaN,<p>you can use the matlab codes for svm and co...,19966.0,2013-01-23 09:00:01,NaN,...,NaN,0,NaN,NaN,NaN,NaN,45118.0,NaN,NaN,NaN
39996,48322,2,NaN,2013-01-23 09:09:34,3,NaN,"<p>I use <a href=""http://www.gnu.org/software/...",892.0,2013-01-23 13:13:30,NaN,...,NaN,2,NaN,892.0,2013-01-23 13:13:30,NaN,48311.0,NaN,NaN,NaN
39997,48323,2,NaN,2013-01-23 09:16:44,1,NaN,"<p>If I understand your question correctly, yo...",2020.0,2013-01-23 09:16:44,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48247.0,NaN,NaN,NaN
39998,48324,2,NaN,2013-01-23 09:36:07,3,NaN,"<p>Doesn't really help you with your question,...",19914.0,2013-01-23 09:36:07,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48297.0,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [6]:
don = pd.DataFrame(ustb)
dona = don[["userId", "Reputation", "Views", "UpVotes", "DownVotes"]]

pstb = pd.DataFrame(posts_tab)
pstba = pstb[["postId", "Score", "userId", "ViewCount", "CommentCount"]]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [7]:
both = dona.merge(pstba)
both.tail()

both.shape

(38962, 9)

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [8]:
both.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [9]:
#there's too many missing values, 
#therefore if I decide to drop any row or column I'm gonna be dismissing
#a lot of information. Maybe fill them with 0 on some missing cells
#and/or median in others, depending on the type of info?



#We could, for each column (or all at once), fill the Nan values with 0 for example - depending on the category
both['ViewCount'] = both['ViewCount'].fillna(0)

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [10]:
both.dtypes

userId            int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
postId            int64
Score             int64
ViewCount       float64
CommentCount      int64
dtype: object

In [11]:
#Turning this column into a different type - int

both["ViewCount"] = both["ViewCount"].astype(int)

In [13]:
both.shape

AttributeError: 'DataFrame' object has no attribute 'dtype'